In [1]:
import numpy as np
import pandas as pd
from bokeh.charts import Bar, output_file, show, hplot, Scatter
import psycopg2
conn = psycopg2.connect(database="taxi", user="postgres", password="guest", 
                        host="ec2-54-197-5-212.compute-1.amazonaws.com", port="5432")

### This section visualizes the types of places the block id is located by showing the counts of various places

In [16]:
cur = conn.cursor()
cur.execute("select * from google_places_type where block_id=899;")
output=cur.fetchall()
conn.commit()

colnames = ['block_id', 'lodging','clothing_store','cafe','restaurant','school','health','meal_delivery',
'point_of_interest','subway_station','electronics_store','home_goods_store','bar','police','convenience_store','store',
'synagogue','furniture_store','food','sublocality_level_1','florist','bicycle_store','locality','hardware_store','night_club',
'park','grocery_or_supermarket','general_contractor','pharmacy','church','jewelry_store','bakery','meal_takeaway','pet_store',
'shoe_store','post_office','book_store','neighborhood','art_gallery','finance','gym','lawyer','doctor','insurance_agency',
'real_estate_agency','accounting','car_dealer','physiotherapist','travel_agency','liquor_store','parking','bus_station',
'atm','hospital','spa','movie_theater','library','museum','moving_company','cemetery','storage','local_government_office',
'car_repair','transit_station','courthouse','gas_station','dentist','department_store','car_rental','hindu_temple','funeral_home',
'painter','car_wash','bank','mosque','locksmith','university','hair_care','beauty_salon','shopping_mall','place_of_worship',
'roofing_contractor','plumber','laundry','veterinary_care','electrician','amusement_park','stadium','movie_rental','campground',
'bowling_alley','premise','natural_feature','zoo','aquarium','light_rail_station','rv_park','embassy','city_hall','airport',
'fire_station','casino']
df = pd.DataFrame(output, columns=colnames) ##initialize pandas dataframe

df_melt = pd.melt(df, id_vars=['block_id'], var_name='Type', value_name='Count') ##conver it to stacked 
block_id = df_melt['block_id'][0] ##name for chart
df_melt = df_melt[df_melt['Count'] >= 0]

ignore_type = ['locality', 'sublocality_level_1', 'point_of_interest', 'sublocality', 'political', 'establishment']

df_melt = df_melt[~df_melt['Type'].isin(ignore_type)] ##remove the "uninteresting" types

In [17]:
chart_title = 'Types of Places for Block ID ' + str(block_id)

bar = Bar(df_melt[['Type', 'Count']], 'Type', values='Count', title=chart_title, width=800, legend=None)
bar.logo = None
bar.toolbar_location = None
show(bar)

### This section visualizes the relationship between Precipitation Amount and Total Tip Across Months and Years

In [21]:
cur = conn.cursor()
cur.execute("select extract(year from date(datetime)) as year, extract(month from date(datetime)) as month, sum(precip_intensity) as sum_precip from weather group by extract(year from date(datetime)), extract(month from date(datetime));")
output=cur.fetchall()
conn.commit()

weather = pd.DataFrame(output, columns=['Year', 'Month', 'Total_Precipitation'])

cur = conn.cursor()
cur.execute("select block_id, extract(year from date) as year, extract(month from date) as month, tips/fares from trips_by_month where block_id = 500")
output=cur.fetchall()
conn.commit()

by_month = pd.DataFrame(output, columns = ['Block_id', 'Year', 'Month', 'Tip_Fair_ratio'])

In [23]:
block_id = by_month['Block_id'][0] ##name for chart
tip_fair = by_month.merge(weather, left_on=['Year', 'Month'], right_on=['Year','Month']).sort_values(['Year', 'Month'])
tip_fair['Year']=tip_fair['Year'].astype(int)
tip_fair['Total_Precipitation'] = np.log(tip_fair['Total_Precipitation'])
chart_title = 'Tip to Fair Ratio for Block ID ' + str(block_id) + ' vs Total Precipitation'

p = Scatter(tip_fair, x='Total_Precipitation', y = 'Tip_Fair_ratio', marker='Year', color='Year', legend="top_right",
         xlabel='Log(Precipitation)', ylabel='Tip to Fair Ratio', title=chart_title)
p.logo = None
p.toolbar_location = None
show(p)